In [ ]:
# Standard library
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner, _prev_result_color
from thejoker.sampler import tensor_vector_scalar, marginal_ln_likelihood, design_matrix

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
# high-eccentricity orbit with reasonable or randomly chosen parameters
opars = OrbitalParams(P=1.412*u.year, K=1200.205*u.m/u.s, ecc=0., # like Jupiter
                      omega=np.random.uniform(0, 2*np.pi)*u.rad,
                      phi0=np.random.uniform(0, 2*np.pi)*u.rad,
                      v0=np.random.normal(0, 30) * u.km/u.s, 
                      jitter=0.*u.m/u.s)
orbit = opars.rv_orbit(0)
print("Mass function:", opars.mf)
print("  (companion mass: {})".format((opars.mf.value**(1/3.) * u.Msun).to(u.Mjup)))
print("asini:", opars.asini)
print("omega:", opars.omega.to(u.degree))
print("phi0:", opars.phi0.to(u.degree))
print("v0:", opars.v0.to(u.km/u.s))

n_obs = 100 # MAGIC NUMBER: number of observations
# bmjd = np.random.uniform(0, 3*365, size=n_obs) + 55555. # 3 year survey
bmjd = np.linspace(0, 3*365, n_obs) # 3 year survey
bmjd.sort()
rv = orbit.generate_rv_curve(bmjd)
rv_err = np.random.uniform(100, 200, size=n_obs) * u.m/u.s # apogee-like
# rv_err = np.random.uniform(1, 2, size=n_obs) * u.m/u.s # apogee-like
rv = np.random.normal(rv.to(u.m/u.s).value, rv_err.value) * u.m/u.s

data = RVData(t=bmjd, rv=rv, stddev=rv_err)
data.plot(marker='.', rv_unit=u.km/u.s)

In [ ]:
nonlinear_p = np.vstack((opars._P, opars._phi0, opars._ecc, opars._omega, 0)).T[0]

ivar = data.get_ivar(0.)
A = design_matrix(nonlinear_p, data._t, data.t_offset)
ATCinvA,p,chi2 = tensor_vector_scalar(A, ivar, data._rv)
chi2

In [ ]:
s2 = (1E3)**2
nonlinear_p = np.vstack((opars._P, 
                         opars._phi0, 
                         opars._ecc, 
                         opars._omega, 
                         s2)).T[0]

A = design_matrix(nonlinear_p, data._t, data.t_offset)
# print(A)

ivar = data.get_ivar(s2)
ATCinv = (A.T * ivar[None])
ATCinvA = ATCinv.dot(A)

# def func(p):
#     return (A.dot(p) - data._rv) * ivar
# print(leastsq(func, x0=[0., 100.])[0])

# Note: this is unstable! if cond num is high, could do:
p = np.linalg.solve(ATCinvA, ATCinv.dot(data._rv))
print(p)
print(np.sqrt(np.diag(np.linalg.inv(ATCinvA))))

In [ ]:
plt.plot(data._t, data._rv, ls='none')
plt.plot(data._t, A.dot(p), marker='')

In [ ]:
s2 = np.logspace(-2, 4, 128)**2
logdet = np.zeros_like(s2)
exp_arg = np.zeros_like(s2)
for i,val in enumerate(s2):
    nonlinear_p = np.vstack((opars._P, 
                             opars._phi0, 
                             opars._ecc, 
                             opars._omega, 
                             val)).T[0]
    
    A = design_matrix(nonlinear_p, data._t, data.t_offset)
    ivar = data.get_ivar(val)
    ATCinv = (A.T * ivar[None])
    ATCinvA = ATCinv.dot(A)

    v0_K = np.linalg.solve(ATCinvA, ATCinv.dot(data._rv))

    dy = A.dot(v0_K) - data._rv
    chi2 = np.sum(dy**2 * ivar) # don't need log term for the jitter b.c. in likelihood below
    
    # ATCinvA,p,chi2 = tensor_vector_scalar(nonlinear_p, data)
    logdet[i] = np.linalg.slogdet(ATCinvA/(2*np.pi))[1] + np.sum(np.log(ivar/(2*np.pi)))
    exp_arg[i] = -0.5*chi2
    
fig,ax = plt.subplots(1,1)
ax.semilogx(np.sqrt(s2), logdet, c='r', marker='', label='logdet')
ax.semilogx(np.sqrt(s2), exp_arg, c='g', marker='', label='-0.5*chi2')
ax.semilogx(np.sqrt(s2), exp_arg + 0.5*logdet, c='k', marker='', lw=2)
ax.axvline(np.median(rv_err.value), color='b', label='median uncertainty')
ax.legend(loc='center left')
ax.set_xlabel('$s$ [m/s]')

In [ ]:
s2 = np.logspace(-2, 4, 128)**2
ll = np.zeros_like(s2)
for i,val in enumerate(s2):
    nonlinear_p = np.vstack((opars._P, 
                             opars._phi0, 
                             opars._ecc, 
                             opars._omega, 
                             val)).T[0]
#     ATCinvA,p,chi2 = tensor_vector_scalar(nonlinear_p, data)
#     logdet = np.linalg.slogdet(ATCinvA)[1]
#     ll[i] = -0.5*np.atleast_1d(chi2) + 0.5*logdet - 0.5*np.sum(np.log(rv_err.value**2 + val))
    ll[i] = marginal_ln_likelihood(nonlinear_p, data)
    
fig,axes = plt.subplots(1,2,figsize=(12,5))
axes[0].semilogx(np.sqrt(s2), ll, c='k')
axes[1].semilogx(np.sqrt(s2), np.exp(ll - ll.max()), c='k')
axes[1].set_ylim(0,1)

axes[0].axvline(np.median(rv_err.value))
axes[1].axvline(np.median(rv_err.value))